# BLAST Proteins

Execução do BLAST através do Biopython no ambiente jupyter.

In [1]:
from Bio.Blast.Applications import NcbimakeblastdbCommandline
from Bio.Blast.Applications import NcbiblastpCommandline

In [2]:
#Criação do banco
cLine = NcbimakeblastdbCommandline(dbtype="prot", title="goodProteins", input_file="goodProteins.fasta")

In [3]:
cLine

NcbimakeblastdbCommandline(cmd='makeblastdb', dbtype='prot', input_file='goodProteins.fasta', title='goodProteins')

In [4]:
print(cLine)

makeblastdb -dbtype prot -in goodProteins.fasta -title goodProteins


In [5]:
stdout, stderr = cLine()

In [6]:
# Execução do BLASTP
cLineProtein = NcbiblastpCommandline(query="goodProteins.fasta", db="goodProteins.fasta",
                                     evalue=1e-5, outfmt=6, out="resultado_tabular")

In [7]:
print(cLineProtein)

blastp -out resultado_tabular -outfmt 6 -query goodProteins.fasta -db goodProteins.fasta -evalue 1e-05


In [8]:
stdout, stderr = cLineProtein()

In [9]:
# Execução do BLASTP (Formato CSV)
cLineProteinCSV = NcbiblastpCommandline(query="goodProteins.fasta", db="goodProteins.fasta",
                                     evalue=1e-5, outfmt=10, out="resultado.csv")

In [10]:
stdout, stderr = cLineProteinCSV()

# Aplicação do Spark para leitura de arquivo tabular como csv

In [11]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder.appName('blast').getOrCreate()

In [36]:
blastDF = spark.read.csv("resultado_tabular", sep="\t")
#peopleDF.select("name", "age").write.format("parquet").save("namesAndAges.parquet")
blastDF

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string]

In [37]:
blastDF.show()

+-----------+------------+-------+---+---+---+---+---+---+---+---------+----+
|        _c0|         _c1|    _c2|_c3|_c4|_c5|_c6|_c7|_c8|_c9|     _c10|_c11|
+-----------+------------+-------+---+---+---+---+---+---+---+---------+----+
|sac|YPL071C| sac|YPL071C|100.000|156|  0|  0|  1|156|  1|156|8.30e-115| 320|
|sac|YLL050C| sac|YLL050C|100.000|143|  0|  0|  1|143|  1|143|2.53e-104| 293|
|sac|YLL050C|lei|CBZ12381| 31.884|138| 87|  5|  1|136|  1|133| 1.83e-14|65.1|
|sac|YMR172W| sac|YMR172W|100.000|719|  0|  0|  1|719|  1|719|      0.0|1471|
|sac|YMR172W| sac|YOL116W| 39.000|100| 57|  2|617|715|278|374| 1.56e-16|81.3|
|sac|YOR185C| sac|YOR185C|100.000|220|  0|  0|  1|220|  1|220|8.56e-169| 462|
|sac|YOR185C| sac|YLR293C| 97.727|220|  4|  1|  1|220|  1|219|1.07e-162| 447|
|sac|YOR185C|lei|CAJ05272| 69.082|207| 64|  0| 11|217|  9|215|1.43e-107| 307|
|sac|YOR185C|lei|CAJ02862| 35.185|162|102|  2| 13|171|  7|168| 1.15e-30| 111|
|sac|YOR185C| sac|YOR089C| 34.118|170|106|  3|  8|171|  2|171| 5

## Aplicação do Spark para leitura de arquivo CSV e conversão para o formato parquet

In [42]:
blastDF.write.parquet("resultado.parquet")

In [43]:
parquetFileDF = spark.read.parquet("resultado.parquet")

In [44]:
parquetFileDF.show()

+-----------+------------+-------+---+---+---+---+---+---+---+---------+----+
|        _c0|         _c1|    _c2|_c3|_c4|_c5|_c6|_c7|_c8|_c9|     _c10|_c11|
+-----------+------------+-------+---+---+---+---+---+---+---+---------+----+
|sac|YPL071C| sac|YPL071C|100.000|156|  0|  0|  1|156|  1|156|8.30e-115| 320|
|sac|YLL050C| sac|YLL050C|100.000|143|  0|  0|  1|143|  1|143|2.53e-104| 293|
|sac|YLL050C|lei|CBZ12381| 31.884|138| 87|  5|  1|136|  1|133| 1.83e-14|65.1|
|sac|YMR172W| sac|YMR172W|100.000|719|  0|  0|  1|719|  1|719|      0.0|1471|
|sac|YMR172W| sac|YOL116W| 39.000|100| 57|  2|617|715|278|374| 1.56e-16|81.3|
|sac|YOR185C| sac|YOR185C|100.000|220|  0|  0|  1|220|  1|220|8.56e-169| 462|
|sac|YOR185C| sac|YLR293C| 97.727|220|  4|  1|  1|220|  1|219|1.07e-162| 447|
|sac|YOR185C|lei|CAJ05272| 69.082|207| 64|  0| 11|217|  9|215|1.43e-107| 307|
|sac|YOR185C|lei|CAJ02862| 35.185|162|102|  2| 13|171|  7|168| 1.15e-30| 111|
|sac|YOR185C| sac|YOR089C| 34.118|170|106|  3|  8|171|  2|171| 5

In [45]:
# Parquet files can also be used to create a temporary view and then used in SQL statements
parquetFileDF.createOrReplaceTempView("parquetFile")

In [46]:
'''
    Nessa primeira consulta, ela filtra todas as linhas em que se encontra a proteína
    'sac|YOR185C' na coluna _c0 e _c1
'''
query = spark.sql("SELECT * FROM parquetFile WHERE _c0 = 'sac|YPL071C' OR _c1 = 'sac|YPL071C'")
query.show() # Para mostrar as primeiras linhas

+-----------+-----------+-------+---+---+---+---+---+---+---+---------+----+
|        _c0|        _c1|    _c2|_c3|_c4|_c5|_c6|_c7|_c8|_c9|     _c10|_c11|
+-----------+-----------+-------+---+---+---+---+---+---+---+---------+----+
|sac|YPL071C|sac|YPL071C|100.000|156|  0|  0|  1|156|  1|156|8.30e-115| 320|
+-----------+-----------+-------+---+---+---+---+---+---+---+---------+----+



In [47]:
# Mostra todas linhas, além de coletar para criar outra base
query.collect()

[Row(_c0='sac|YPL071C', _c1='sac|YPL071C', _c2='100.000', _c3='156', _c4='0', _c5='0', _c6='1', _c7='156', _c8='1', _c9='156', _c10='8.30e-115', _c11='320')]

In [48]:
'''
    Nessa segunda consulta, ela conta todas as linhas em que a coluna _c4 = 200
'''
query = spark.sql("SELECT count(*)FROM parquetFile WHERE _c4 = 200")
query.show() # Para mostrar as primeiras linhas

+--------+
|count(1)|
+--------+
|     294|
+--------+



In [49]:
'''
    Nessa terceira consulta, ela conta todas as linhas em que a coluna _c4 = 200 e agrupa 
    pela coluna _c0
'''
query = spark.sql("SELECT _c0, count(_c4) FROM parquetFile WHERE _c4 = 200 GROUP BY _c0")
query.show() # Para mostrar as primeiras linhas

+------------+----------+
|         _c0|count(_c4)|
+------------+----------+
|lei|CAJ05207|         1|
|lei|CAJ05828|         1|
|lei|CAJ03416|         1|
|lei|CAJ06470|         1|
| sac|YPL226W|         1|
|lei|CAJ03906|         2|
|lei|CAJ06341|         1|
| sac|YHR001W|         1|
|lei|CAJ08631|         1|
| sac|YPL256C|         1|
|lei|CBZ12160|         1|
| sac|YDL101C|         1|
|lei|CAJ04177|         2|
| sac|YDL119C|         1|
|lei|CAJ04492|         4|
| sac|YFL039C|         1|
|lei|CAJ06500|         1|
|lei|CAJ09455|         1|
| sac|YLL024C|         1|
|lei|CAJ05381|         2|
+------------+----------+
only showing top 20 rows

